In [4]:
import os
import numpy as np
import tensorflow as tf
from pathlib import Path
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# --- KONFIGURASI ---
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 20
DATA_DIR_TRAIN = Path("../data/processed/Train")
DATA_DIR_VAL   = Path("../data/processed/val")

# --- 1. FUNGSI PEMBACA PATH DAN LABEL ---
def get_image_paths_and_labels(directory):
    image_paths = []
    labels = []
    # Ambil nama kelas dari folder
    class_names = sorted([d.name for d in directory.iterdir() if d.is_dir()])
    # Buat mapping kelas ke angka (0, 1, 2...)
    class_to_idx = {cls: i for i, cls in enumerate(class_names)}
    
    print(f"Scanning {directory}...")
    for class_name in class_names:
        class_dir = directory / class_name
        # Ambil semua file gambar
        for img_path in class_dir.glob("*"):
            if img_path.suffix.lower() in ['.jpg', '.jpeg', '.png', '.bmp']:
                image_paths.append(str(img_path))
                labels.append(class_to_idx[class_name])
                
    return image_paths, labels, class_names

# Ambil list path file dan labelnya
train_paths, train_labels, classes = get_image_paths_and_labels(DATA_DIR_TRAIN)
val_paths, val_labels, _ = get_image_paths_and_labels(DATA_DIR_VAL)

print(f"Jumlah Data Train: {len(train_paths)}")
print(f"Jumlah Data Val: {len(val_paths)}")
print(f"Jumlah Kelas: {len(classes)}")

if len(train_paths) == 0:
    raise ValueError("Tidak ada gambar ditemukan! Pastikan path '../data/processed/Train' benar.")

Scanning ..\data\processed\Train...
Scanning ..\data\processed\val...
Jumlah Data Train: 265
Jumlah Data Val: 0
Jumlah Kelas: 70


In [9]:
# Fungsi ini akan dijalankan TensorFlow secara otomatis untuk setiap gambar
def load_and_process_image(path, label):
    # 1. Baca file dari disk
    img = tf.io.read_file(path)
    # 2. Decode gambar (ubah dari bytes ke tensor gambar)
    img = tf.io.decode_image(img, channels=3, expand_animations=False)
    # 3. Pastikan ukuran sama
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    # 4. Set shape eksplisit (PENTING untuk menghindari error shape=(None,))
    img.set_shape([IMG_SIZE, IMG_SIZE, 3])
    # 5. Preprocessing khusus MobileNetV2
    img = tf.keras.applications.mobilenet_v2.preprocess_input(img)
    return img, label

# Buat Dataset Pipeline dari file path (pastikan train_paths dan train_labels bertipe list of str dan int)
train_ds = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
train_ds = train_ds.map(load_and_process_image, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Pastikan val_paths dan val_labels adalah list of str dan int
if len(val_paths) > 0 and len(val_labels) > 0:
    val_ds = tf.data.Dataset.from_tensor_slices((val_paths, val_labels))
    val_ds = val_ds.map(load_and_process_image, num_parallel_calls=tf.data.AUTOTUNE)
    val_ds = val_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
else:
    val_ds = None
    print("Warning: Validation data is empty. val_ds is set to None.")

In [10]:
# Augmentasi data
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
])

# Base Model
base_model = MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False 

inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = data_augmentation(inputs)
x = base_model(x, training=False)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
# Output layer sesuai jumlah kelas
outputs = Dense(len(classes), activation='softmax')(x)

model = Model(inputs, outputs)

model.compile(
    optimizer=Adam(learning_rate=1e-3),
    # Gunakan 'sparse' karena label kita berupa angka integer (bukan one-hot)
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
)

In [11]:
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True, monitor='val_accuracy'),
    ReduceLROnPlateau(patience=3, factor=0.5, min_lr=1e-6, monitor='val_loss'),
    ModelCheckpoint("best_model.keras", save_best_only=True, monitor='val_accuracy')
]

print("\nMulai Training...")
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks
)


Mulai Training...
Epoch 1/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 654ms/step - accuracy: 0.0182 - loss: 4.9654

C:\Users\ACER\AppData\Roaming\Python\Python313\site-packages\keras\src\callbacks\early_stopping.py:99: UserWarning: Early stopping conditioned on metric `val_accuracy` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)
C:\Users\ACER\AppData\Roaming\Python\Python313\site-packages\keras\src\callbacks\callback_list.py:171: UserWarning: Learning rate reduction is conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss,learning_rate.
  callback.on_epoch_end(epoch, logs)
C:\Users\ACER\AppData\Roaming\Python\Python313\site-packages\keras\src\callbacks\model_checkpoint.py:276: UserWarning: Can save best model only with val_accuracy available.
  if self._should_save_model(epoch, batch, logs, filepath):


9/9 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.0189 - loss: 4.9499 - learning_rate: 0.0010
Epoch 2/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 472ms/step - accuracy: 0.0528 - loss: 4.0543 - learning_rate: 0.0010
Epoch 3/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 418ms/step - accuracy: 0.1585 - loss: 3.4389 - learning_rate: 0.0010
Epoch 4/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 425ms/step - accuracy: 0.2679 - loss: 3.0933 - learning_rate: 0.0010
Epoch 5/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 436ms/step - accuracy: 0.3962 - loss: 2.6759 - learning_rate: 0.0010
Epoch 6/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 418ms/step - accuracy: 0.5019 - loss: 2.3563 - learning_rate: 0.0010
Epoch 7/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 422ms/step - accuracy: 0.5698 - loss: 2.1236 - learning_rate: 0.0010
Epoch 8/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 440ms/step - accuracy: 0.7019 - loss: 1.7074 - learning_rate: 0.0010
Epoch 9/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.7057 - loss: 1.6069 - learning_rate: 0.0010
Epoch 10/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 10s 1s

In [12]:
print("\nUnfreezing base model for Fine-Tuning...")
base_model.trainable = True
fine_tune_at = 100
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

model.compile(
    optimizer=Adam(learning_rate=1e-5), # LR sangat kecil untuk fine tuning
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history_fine = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10, 
    callbacks=callbacks
)


Unfreezing base model for Fine-Tuning...
Epoch 1/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 60s 4s/step - accuracy: 0.1472 - loss: 3.8379 - learning_rate: 1.0000e-05
Epoch 2/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - accuracy: 0.1509 - loss: 3.7617 - learning_rate: 1.0000e-05
Epoch 3/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.1774 - loss: 3.5469 - learning_rate: 1.0000e-05
Epoch 4/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 32s 3s/step - accuracy: 0.2075 - loss: 3.3808 - learning_rate: 1.0000e-05
Epoch 5/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 41s 4s/step - accuracy: 0.2302 - loss: 3.1884 - learning_rate: 1.0000e-05
Epoch 6/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 25s 2s/step - accuracy: 0.2491 - loss: 3.0277 - learning_rate: 1.0000e-05
Epoch 7/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 0.2642 - loss: 2.8616 - learning_rate: 1.0000e-05
Epoch 8/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.3094 - loss: 2.8461 - learning_rate: 1.0000e-05
Epoch 9/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 8s 772ms/step - accuracy: 0.3208 - loss: 2